<a href="https://colab.research.google.com/github/Praveenku32k/Agentic_CrewAI/blob/main/L4_tools_customer_outreach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [ ]:
from crewai import Agent, Task, Crew

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [ ]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
inputs = {
    "lead_name": "Generative AI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Praveen Kumar",
    "position": "jr Data Scientist",
    "url":"https://www.linkedin.com/in/praveen-kumar-sjbit/",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Generative AI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about Generative AI to create a comprehensive report for our sales team.

Action: Search the internet
Action Input: {"search_query": "Generative AI company background"} 


Search results: Title: The Generative AI Company | LinkedIn
Link: https://www.linkedin.com/company/the-generative-ai-company
Snippet: The Generative AI Company provides end-to-end experien

Thought: 
I have gathered information on Generative AI's background, key personnel, recent milestones, and potential needs. It's time to compile a comprehensive report for our sales team.

Final Answer: 
Generative AI is a company that provides end-to-end experience design and technology consulting services. The company has a cloud architecture optimized for cognitive elements in operations workflows, deployable on various cloud platforms, and capable of connecting to multiple AI systems. The history of Generative AI dates back to the 1960s, with early examples like the ELIZA chatbot.

Key personnel at Generative AI include a team leader overseeing 551 people, with direct reports from both research and product teams. The team includes individuals with expertise in managing employees and guiding the development, training, and testing of AI models.

Recent milestones in Generative AI's development include breakthrough innovations in machine learning, the transformative power of large gen

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

Email Draft for Praveen Kumar, Jr Data Scientist at Generative AI:

Subject: Enhancing Generative AI's Recent Product Launch with CrewAI's Solutions

Dear Praveen Kumar,

I hope this email finds you well. I recently came across Generative AI's major product launch and partnerships, and I was truly impressed by the innovative strides your team has made in the AI industry. As a Jr Data Scientist at Generative AI, your contributions to these achievements must be significant, and I wanted to reach out to explore how CrewAI's solutions can support your goals further.

At CrewAI, we specialize in providing end-to-end experience design and technology consulting services tailored to organizations like Generative AI. Our cloud architecture, optimized for cognitive elements in operations workflows, is deployable on various platforms and capable of connecting to multiple AI systems - a perfect match for the innovative work Generative AI is doing.

I believe that our expertise in optimizing cognitive elements, enhancing workforce capabilities, and driving innovation in AI applications aligns perfectly with Generative AI's mission and vision. By leveraging our solutions, you can enhance human capabilities through AI-based tools, transform HR roles, and drive innovation in ways that will shape the future of artificial intelligence.

I would love the opportunity to discuss further how CrewAI can support Generative AI in achieving your goals and unlocking new possibilities in the AI industry. Please let me know a convenient time for a call or meeting to explore this potential collaboration.

Thank you for your time, Praveen. I look forward to the opportunity to connect and explore how CrewAI can contribute to the continued success of Generative AI.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Email Draft for Praveen Kumar, Jr Data Scientist at Generative AI:

Subject: Elevating Generative AI's Innovations with CrewAI's Solutions

Dear Praveen Kumar,

I trust you are doing well. I recently came across the groundbreaking innovations in machine learning and large generative models that Generative AI has achieved, and I must say, the transformative power of your work is truly inspiring. As a Jr Data Scientist at Generative AI, your role in shaping the future of artificial intelligence is commendable, and I believe CrewAI's solutions can further support your efforts.

CrewAI specializes in delivering end-to-end experience design and technology consulting services that are perfectly aligned with the goals of innovative companies like Generative AI. Our cloud architecture, designed to optimize cognitive elements in operations workflows, can seamlessly integrate with various platforms and connect to multiple AI systems - offering a robust solution for the cutting-edge work you are doing.

I am confident that our capabilities in enhancing workforce capabilities, optimizing cognitive elements, and driving innovation in AI applications can complement and elevate the incredible work Generative AI is doing. By leveraging CrewAI's solutions, you can enhance human capabilities through AI-based tools, transform traditional HR roles, and influence jobs in ways that were once thought immune to automation.

I would be delighted to discuss how CrewAI can support Generative AI in realizing your goals and driving impactful change in the AI industry. Please let me know a suitable time for a call or meeting to explore this potential partnership further.

Thank you for your time, Praveen. I am excited about the possibility of collaborating with Generative AI and contributing to your continued success.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI